<code>pip install retina-face</code>


In [24]:
import cv2
import os
import numpy as np

In [25]:
def emotionImage(emotion):
	# Emojis
	if emotion == 'Felicidad': 	image = cv2.imread('./Emojis/Felicidad.png', cv2.IMREAD_UNCHANGED)
	if emotion == 'Enojo': 		image = cv2.imread('./Emojis/Enojado.png', cv2.IMREAD_UNCHANGED)
	if emotion == 'Sorpresa': 	image = cv2.imread('./Emojis/Sorpresa.png', cv2.IMREAD_UNCHANGED)
	if emotion == 'Tristeza': 	image = cv2.imread('./Emojis/Tristeza.png', cv2.IMREAD_UNCHANGED)
	if emotion == 'Neutro': 	image = cv2.imread('./Emojis/Neutro.png', cv2.IMREAD_UNCHANGED)
	return image

In [26]:
emotion_recognizer = cv2.face.LBPHFaceRecognizer_create()
emotion_recognizer.read('./Modelos/modeloLBPH.xml')

In [27]:
dataPath = './fotos_entrenamiento/'
imagePaths = os.listdir(dataPath)
print('imagePaths=',imagePaths)

imagePaths= ['Enojo', 'Felicidad', 'Neutro', 'Sorpresa', 'Tristeza']


In [28]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else: print('Camera 0')
    else: print('Camera 1')
else: print('Camera 0')

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

while True:

	ret,frame = cap.read()
	if ret == False: break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	auxFrame = gray.copy()
 
	faces = faceClassif.detectMultiScale(gray,
									scaleFactor=1.1,
									minNeighbors=3,
									minSize=(5, 5),
								)

	for (x,y,w,h) in faces:
		rostro = auxFrame[y:y+h,x:x+w]
		rostro = cv2.resize(rostro,(150,150),interpolation= cv2.INTER_CUBIC)
		result = emotion_recognizer.predict(rostro)

		#cv2.putText(frame,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
		resized = cv2.resize(image, (w, h), interpolation = cv2.INTER_AREA)
		filas = resized.shape[0]
		col = w
		alto = filas // 3
		if result[1] < 60:
			#print(result)
			if(result[0] is 0): color = (50, 50, 255)
			elif(result[0] is 1): color = (50, 255, 100)
			elif(result[0] is 2): color = (255, 255, 255)
			elif(result[0] is 3): color = (255, 255, 75)
			elif(result[0] is 4): color = (255, 50, 75)
   
			#Emoticono
			n_frame = frame[y - h + alto:y + alto, x:x + col]
			image = resized[:, :, 3]
			image_inv = cv2.bitwise_not(image)
                
			bg_black = cv2.bitwise_and(resized, resized, mask = image)
			bg_black = bg_black[:, :, 0:3]
			bg_frame = cv2.bitwise_and(n_frame, n_frame, mask = image_inv)
                    
			result = cv2.add(bg_black, bg_frame)
			frame[y-filas+alto:y+alto, x:x+col] = result
   
			cv2.putText(frame,'{}'.format(imagePaths[result[0]]),(x,y-10),2,1.1,(color),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),color,2)
			#nFrame = cv2.hconcat([frame,image])
		else:
			cv2.putText(frame,'No identificado',(x,y-20),2,0.8,(0,0,255),1,cv2.LINE_AA)
			cv2.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
			#nFrame = cv2.hconcat([frame,np.zeros((480,300,3),dtype=np.uint8)])

	cv2.imshow('Vigila tus sentimientos',frame)
	k = cv2.waitKey(1)
	if k == 27:
		break

cap.release()
cv2.destroyAllWindows()

Camera 0


NameError: name 'image' is not defined